전처리

In [2]:
import warnings
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import mplcursors # Use this is for creating a cursor-interactive plot with "%matplotlib notebook"
from sklearn.decomposition import NMF # Use this for training Non-negative Matrix Factorization
from sklearn.utils.extmath import randomized_svd # Use this for training Singular Value Decomposition
from sklearn.manifold import TSNE # Use this for training t-sne manifolding

plt.style.use('ggplot') # You can also use different style

# just for plot checking, use this option
# %matplotlib inline

# for interactive plot
# If you use this option, plot will appear at first-drawn position
%matplotlib notebook

warnings.filterwarnings('ignore')

데이터셋 불러오기
데이터셋 출처 https://www.kaggle.com/datasets/deepcontractor/top-video-games-19952021-metacritic
데이터셋 1995년부터 2021년까지의 자료
all_games.csv의 칼럼
names platform release_date summary(게임 요약) meta_score(전문가 스코어 100점만점) user_review(유저스코어 10점만점)


In [39]:
dir = './dataset/'
df_games=pd.read_csv(dir+'all_games.csv',usecols=['name','platform','release_date','meta_score','user_review'])

게임수 파악
6546개의 플랫폼이 다른 동일 게임이 존재 

In [40]:
n_games=len(df_games['name'])
n_game_same_name=len(df_games['name'].unique())
n_games,n_game_same_name,n_games-n_game_same_name

(18800, 12254, 6546)

이름이없는 row 존재하지않음

In [41]:
len(df_games)-n_games

0

칼럼 타입 확인하기

In [42]:
df_games.dtypes

name            object
platform        object
release_date    object
meta_score       int64
user_review     object
dtype: object

user_review칼럼에서 tbd(미정)값 존재 확인

In [43]:
df_games.astype({'name':'string','platform':'string','release_date':'string','user_review':'string'})

,name,platform,release_date,meta_score,user_review
0,The Legend of Zelda: Ocarina of Time,Nintendo 64,"November 23, 1998",99,9.1
1,Tony Hawk's Pro Skater 2,PlayStation,"September 20, 2000",98,7.4
2,Grand Theft Auto IV,PlayStation 3,"April 29, 2008",98,7.7
3,SoulCalibur,Dreamcast,"September 8, 1999",98,8.4
4,Grand Theft Auto IV,Xbox 360,"April 29, 2008",98,7.9
...,...,...,...,...,...
18795,Fast & Furious: Showdown,Xbox 360,"May 21, 2013",22,1.3
18796,Drake of the 99 Dragons,Xbox,"November 3, 2003",22,1.7
18797,Afro Samurai 2: Revenge of Kuma Volume One,PlayStation 4,"September 22, 2015",21,2.9
18798,Infestation: Survivor Stories (The War Z),PC,"October 15, 2012",20,1.7


user review칼럼에서 tbd값이 포함된 row처리?

일단 0.0점으로 변경(알고리즘에 따라 달라질듯)

In [4]:
df_games.loc[df_games.user_review=='tbd','user_review']='0.0'

NameError: name 'df_games' is not defined

tbd값 처리후 데이터타입 float으로 변경

In [45]:
df_games.astype({'user_review':'float'})

,name,platform,release_date,meta_score,user_review
0,The Legend of Zelda: Ocarina of Time,Nintendo 64,"November 23, 1998",99,9.1
1,Tony Hawk's Pro Skater 2,PlayStation,"September 20, 2000",98,7.4
2,Grand Theft Auto IV,PlayStation 3,"April 29, 2008",98,7.7
3,SoulCalibur,Dreamcast,"September 8, 1999",98,8.4
4,Grand Theft Auto IV,Xbox 360,"April 29, 2008",98,7.9
...,...,...,...,...,...
18795,Fast & Furious: Showdown,Xbox 360,"May 21, 2013",22,1.3
18796,Drake of the 99 Dragons,Xbox,"November 3, 2003",22,1.7
18797,Afro Samurai 2: Revenge of Kuma Volume One,PlayStation 4,"September 22, 2015",21,2.9
18798,Infestation: Survivor Stories (The War Z),PC,"October 15, 2012",20,1.7


칼럼추가

데이터프레임 csv로 변환

In [2]:
df_games.to_csv(dir+'all_games_proc.csv',sep=',',float_format='%.2f',index=False)

NameError: name 'df_games' is not defined

In [22]:
dir2="../data/"
df_games2=pd.read_csv(dir2+'ps4.csv')
df_games2

,genres,metascore,developer,userscore,rating,publisher,title,summary,release_date
0,Simulation|Virtual|Career|,43,Bossa Studios,4.4,T,Bossa Studios,Surgeon Simulator: Experience Reality,This game is ridiculously fun once you've plac...,"Dec 3, 2016"
1,Action Adventure|Sci-Fi|General|,43,Edge of Reality,4.2,T,Activision,Transformers: Rise of the Dark Spark,Fight your way through both Earth and Cybertro...,"Jun 24, 2014"
2,Arcade|Sports|Traditional|Team|Basketball|Arcade|,43,EA Sports,2.6,E,Electronic Arts,NBA Live 14,BounceTek -- Revolutionary physics-based dribb...,"Nov 19, 2013"
3,Sports|Individual|Combat|Wrestling|,43,Visual Concepts,1.6,T,2K Games,WWE 2K20,F the haters. This game is still fun. And it’...,"Oct 22, 2019"
4,Racing|Arcade|Other|,42,Viewpoint Games,5.6,E,Viewpoint Games,VR Karts,The checkered flag is about to drop on a new V...,"Sep 12, 2017"
...,...,...,...,...,...,...,...,...,...
2633,Action|Shooter|First-Person|Arcade|,87,MachineGames,7.3,M,Bethesda Softworks,Wolfenstein II: The New Colossus,Wolfenstein II: The New Colossus is the highly...,"Oct 27, 2017"
2634,Role-Playing|Action RPG|,87,Vanillaware,8.2,T,Atlus,Odin Sphere Leifthrasir,The tale of Armageddon is about to be told... ...,"Jun 7, 2016"
2635,Role-Playing|Action RPG|,87,From Software,7.7,T,Bandai Namco Games,Dark Souls II: Scholar of the First Sin,Dark Souls II: Scholar of the First Sin includ...,"Apr 7, 2015"
2636,Action|Platformer|2D|,87,Sega,7.8,E,Sega,Sonic Mania Plus,"With the addition of encore mode, this game is...","Jul 17, 2018"


ps4.csv로 위와같이 하기

In [24]:
df_games2.loc[df_games2.userscore=='tbd','userscore']=df_games2.loc[df_games2.userscore!='tbd']['userscore'].astype({'userscore':float}).describe()['mean']
df_games2=df_games2.astype({'userscore':'float'})

In [26]:
df_games2.to_csv(dir2+'ps4.csv',sep=',',float_format='%.2f',index=False)